In [ ]:
# main.py

import cv2
import face_recognition
import os
import numpy as np
from datetime import datetime
import pandas as pd

# Load known images and encode them
path = 'Images'  # Folder containing reference images
images = []
classNames = []
myList = os.listdir(path)

for imgName in myList:
    img = cv2.imread(f'{path}/{imgName}')
    images.append(img)
    classNames.append(os.path.splitext(imgName)[0])

def findEncodings(images):
    encodeList = []
    for img in images:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        enc = face_recognition.face_encodings(img_rgb)[0]
        encodeList.append(enc)
    return encodeList

encodeListKnown = findEncodings(images)
print("Encoding Complete")

# Create attendance log
def markAttendance(name):
    now = datetime.now()
    dt_string = now.strftime('%Y-%m-%d')
    time_string = now.strftime('%H:%M:%S')

    filename = f"Attendance_{dt_string}.csv"

    # Create CSV if not exists
    if not os.path.exists(filename):
        with open(filename, 'w') as f:
            f.write('Name,Time\n')

    # Read existing data
    df = pd.read_csv(filename)
    if name not in df['Name'].values:
        with open(filename, 'a') as f:
            f.write(f'{name},{time_string}\n')

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    img_small = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
    img_small_rgb = cv2.cvtColor(img_small, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(img_small_rgb)
    encodesCurFrame = face_recognition.face_encodings(img_small_rgb, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            markAttendance(name)

            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4

            cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(img, name, (x1+6, y2+25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    cv2.imshow('Attendance System', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
